# Prerequisites


In [2]:
# Definitions
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')

from matplotlib.animation import FuncAnimation

def animate_dynamic_point(trajectory, resolution_Hz=15, duration=None, fig_size=8):
    plt.ioff()
    trajectory = np.array(trajectory)
    trajectory = trajectory.T[:40].T
    if len(trajectory.shape) == 1:
        trajectory = trajectory.reshape(trajectory.size, 1)
    if trajectory.shape == (trajectory.size, 1):
        trajectory = np.stack((trajectory.T[0], np.zeros(trajectory.size))).T
    if duration == None:
        frames = range(trajectory.shape[0])
    else:
        frames = range(int(duration * resolution_Hz))
    fig, ax = plt.subplots(figsize=(fig_size, fig_size))
    # set the axes limits
    ax.axis([-2,2,-2, 2])
    ax.set_aspect("equal")
    # create a point in the axes
    plt.grid()
    ax.plot(0, 0, markersize=108, marker="o", color = 'red')

    plt.text(0, 0, 'Target', horizontalalignment='center', verticalalignment='center', color='white')
    points = [ax.plot(0,1, marker="o")[0] for _ in range(20)]

    # Updating function, to be repeatedly called by the animation
    def update(t):
        # obtain point coordinates
        x12 = trajectory[int(t) % trajectory.shape[0]][:40]
        # set point's coordinates
        for i in range(20):
            points[i].set_data([x12[i]],[x12[20 + i]])
        return points


    ani = FuncAnimation(fig, update, interval=1000/resolution_Hz, blit=True, repeat=True,
                    frames=frames)
    plt.ion()
    return ani

# A collection of dynamic bodies
Below is a system that represents a group of 20 point masses moving on a plane with friction. The goal is to drive the masses to a repelling point in the middle of the plane. The agent is capable of pushing the masses either vertically or horizontally, but whenever a mass is being pushed vertically, the same force will be appliead to all other masses horizontally.

$$
\begin{aligned}
&\begin{cases}
\frac{\partial}{\partial t}x_1(t)= x_3(t),\\
\frac{\partial}{\partial t}x_2(t)= x_4(t),\\
\frac{\partial}{\partial t}x_3(t)= x_1(t) - x_3(t) + u_1 + \sum u_2,\\
\frac{\partial}{\partial t}x_4(t)= x_2(t) - x_4(t) + u_2,
\end{cases}\\
& x_1, x_2, x_3, x_4, u_1, u_2 \in \mathbb{R}^{20},\\
& x_1(0), x_2(0), x_3(0), x_4(0) \in [-1, 1]\\
& r(x(t), u(t)) = \lVert x \rVert_2^2 + \lVert u \rVert_2^2, \\
& u(t) = \rho(x(t))\\
\end{aligned}
$$

An 80-dimensional system !? How could we possibly solve such a problem?

# Introduction to Linear-Quadratic Regulators (LQR)

Mathematically speaking, linear systems make up only a tiny subset of possible control systems. However it is also true that in practice you will encounter them **A LOT**. Likewise, qudratic costs (i.e. $r(x, u) = x^TQx + x^TNu + u^TRu$) are capable of phrasing the **vast majority of reasonable goals** in control.

It is important to remember that if you did come across a linear system (and you did manage to phrase your goals using quadratic costs), then you should **never** use RL to solve such a problem or else you will look foolish. Indeed, when it comes to nonlinear systems RL is powerful, but with linear systems it is significantly outperformed by a far more convenient approach called **Linear-Qudratic Regulation**.

For a continuous-time linear system described by:

$$\dot{x} = Ax + Bu$$

with a cost function defined as:

$$J = \int_{0}^\infty \left( x^T Q x + u^T R u + 2 x^T N u \right) dt$$

the feedback control law that minimizes the value of the cost is:

$$u = -K x \,$$

where $K$ is given by:

$$K = R^{-1} (B^T P + N^T) \,$$

and $P$ is found by solving the continuous time [algebraic Ricatti equation](https://en.wikipedia.org/wiki/Algebraic_Riccati_equation) (CARE):

$$A^T P + P A - (P B + N) R^{-1} (B^T P + N^T) + Q = 0 \,$$

This feedback $-Kx$ is known to be globally optimal. Thus we have reduced our optimal control problem to the problem of solving CARE. Fortunately, `scipy` provides highly efficient numerical methods for solving algebraic Ricatti equations, including CARE. See [`scipy.linalg.solve_continuous_are`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_continuous_are.html) for more details.

# Problem 1: Apply LQR to drive the point masses to the target!
(and simulate the outcome)

In [10]:
import numpy as np
import scipy.linalg, scipy.integrate
from scipy.linalg import solve_continuous_are
from scipy.integrate import solve_ivp

n = 20

A = np.kron(
    np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, -1, 0],
        [0, 1, 0, -1],
    ]),
    np.eye(n),
)
B = np.kron(
    np.array([
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 1],
    ]),
    np.eye(n),
)
B[2*n : 3*n, n:] = np.ones((n, n))
Q = np.eye(4 * n)
R = np.eye(2 * n)


P = solve_continuous_are(A, B, Q, R)
K = np.linalg.inv(R) @ B.T @ P


x_0 = np.random.random(size=80) * 2 - 1

def state_dynamics_function(x, u):
    return A@x + B@u

def rho(x):
    # A.k.a feedback
    return -K @ x

def get_trajectory(x_0, t_final=10, frames=40):
    def f(t, x):
        return state_dynamics_function(x, rho(x))
    sol = solve_ivp(f, [0, 10], x_0,
                    t_eval=t_final * np.arange(frames) / frames)
    return sol.y.T

trajectory = get_trajectory(x_0)

animate_dynamic_point(trajectory)